In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('../../src'))
os.environ["OMP_NUM_THREADS"] = "8" # export OMP_NUM_THREADS=4

In [ ]:
from ghost_buster import display_image as display
from ghost_buster import sources_image as sim
from ghost_buster import ghosts_corr as gcorr
from ghost_buster import ghosts_fit as gfit
from ghost_buster import ghosts_simu as gsim

In [ ]:
import pylab as plt
import numpy as np
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwimage
import lsst.afw.fits as afwfits

In [ ]:
ghost_292=afwimage.ImageF.readFits("ghost_292.fits", 0)

In [ ]:
display.displayImage(ghost_292, title="2024111100292")
display.displayImageGhosts(ghost_292, zmin=850, zmax=1100, title="2024111100292")
display.displayImageGhostsBW(ghost_292, title="2024111100292")

In [ ]:
real = ghost_292.getArray()

In [ ]:
display.displayImageGhostsFlux(real)
display.displayReal(real)

In [ ]:
md_292=afwfits.readMetadata("ghost_292.fits")
md_292["RA"], md_292["DEC"], md_292["ROTPA"]

In [ ]:
rm_src = sim.removeSources(real)
display.displayRemoveSources(real, rm_src)

In [ ]:
catalog = sim.getCatalog(real)
x_star, y_star = sim.getBrightnessCoord(catalog)

In [ ]:
theta_x, theta_y = sim.getCoordBatoid(real)

In [ ]:
init_params = gsim.initParams(real, md_292, minflux=1e-5)

In [ ]:
telescope = gsim.initTelescope()

In [ ]:
x_sep, y_sep, flux_sep = gsim.batoidCalcul(telescope, init_params)

In [ ]:
x_grp, y_grp, flux_grp = gsim.groupData(x_sep, y_sep, flux_sep)

In [ ]:
x_grp, y_grp = gsim.rotAfterBatoid(x_grp, y_grp, md_292)

In [ ]:
px, py = ghost_292.getDimensions()[0], ghost_292.getDimensions()[1]
hist, x_hist, y_hist = gsim.getSimuImage(px, py, x_grp, y_grp, flux_grp)
hist = gfit.applyGrid(real, hist)

In [ ]:
display.displaySimu(hist, x_hist, y_hist)

In [ ]:
print(f"Hist shape : {hist.shape}\nImage shape : {real.shape}")

In [ ]:
gfit.showNoise(real)

In [ ]:
clean = gfit.applyFit(real, hist)

In [ ]:
display.displayFit(real, hist, x_hist, y_hist, clean)

Ici, il ne reste plus qu'à faire les corrélations pour sélectionner les ghosts et recommence le fit + plot

Je pense qu'à partir d'ici on peut se concentrer sur les méthodes de fit avant d'approfondir avec des fits aux ordres supérieurs.

In [ ]:
real2, hist2 = sim.removeSourcesBoth(real, hist)
display.displayRemoveSourcesBoth(real2, hist2)

In [ ]:
a = gfit.getFit(real2, hist2)[0]
clean2 = gfit.applyFit2(real, hist, a)

In [ ]:
display.displayFit(real, hist2, x_hist, y_hist, clean2)